## Chapter III [45']

* How to work with friends on the same file... and remain friends (merge)
* Working with remote repositories (clone, fetch, pull, push, pull requests)
* Contributing flows (forks, feature branches)

## How to work with friends on the same file... and remain friends (merge)

Working with other people is always a challenge. We have different brains, perspectives in life, ideologies... there's no need to make it even harder by working on a same file without the right tools!

Luckily git gives us several tools to do that without generating more conflicts than absolutely necessary. One of the set of tools are `git branch`es, the other is the ability to bring work from one branch to another without going bananas.

So let's get our hands dirty. We will create a repository like we did and the last session and add some content to it. 

In [ ]:
%%bash 
rm -Rf my_first_repo
mkdir my_first_repo
git init my_first_repo

rm -f my_first_repo/plot_compare_reduction.py*
# This script (plot_compare_reduction.py) has been obtained via: 
# wget https://scikit-learn.org/stable/_downloads/plot_compare_reduction.py -P resources
# A local copy is held in case sklearn's version changes
cp resources/plot_compare_reduction.py my_first_repo/
cd my_first_repo
git add plot_compare_reduction.py
git commit -m 'checking scikit learn example into our repo'

# We create a first branch:
git branch $USER/experiment

# We create a second branch via: `git checkout -b`, which performs a `branch` + `checkout`
git checkout -b $USER/feature master

# We see that we now have 3 branches and HEAD is pointing at `$USER/feature`
git branch

The three branches right now are pointing at the same (and unique) commit, which means that there are no differences!

In [ ]:
%%bash 
cd my_first_repo
git diff master
git diff arnau.tibau/experiment arnau.tibau/feature
# no output 'cause no difference :)

Next we will work off the two branches that we created (`$USER/experiment` and `$USER/feature`) to introduce differences between the two.

**Exercise 3.0**: You guessed it! Please introduce changes in `plot_compare_reduction.py` in branch `$USER/experiment` and then in `$USER/feature` so that we have differences. **NOTE**: For the sake of this exercise, please introduce differences in **DIFFERENT LINES** for each branch. Just bear with me :)

At this point we should have differring branches:

In [ ]:
%%bash
cd my_first_repo/
# This should show the difference in commits between the two branches
git diff $USER/experiment $USER/feature

In [ ]:
%%bash
cd my_first_repo/
git log --graph

In [ ]:
%%bash
cd my_first_repo/
git checkout $USER/feature
git log --graph

What we will do now is to try to *merge* the changes introduced in these "feature" branches, 
that is, consolidate the progress that we (or the team) made on two independent branches into
one single branch. To this end, we will first create a branch (`$USER/merge_work`) off one of our feature branches (say the one we called `$USER/experiment`), and try to merge the content of the other branch (`$USER/feature`) into this new branch:

In [ ]:
%%bash
cd my_first_repo/
git checkout -b $USER/merge_work $USER/experiment
git merge $USER/feature

If no conflicts arose, git was able to merge the two branches and has created a special merge commit with two ancestors: the last commit in one branch and the last commit in the other:

In [ ]:
%%bash
cd my_first_repo/
git log --graph -n 10

Amazing! If your branch work had no conflicts, you now have a branch `$USER/merge_work` with all the work you've been doing, which you can now test before merging it into the `master` branch: (**If your merge had conflicts, please continue to the next section**)

In [ ]:
%%bash
cd my_first_repo/
# let's test the changes we introduced:
python plot_compare_reduction.py

At this point you may have run into an error.... If you introduced your own changes, perhaps your code ran perfectly and you are good to go. If you did get an error, it turns out we are not done just yet... Which is good news because we're now **learning an important lesson: having no merge conflics doesn't mean that you haven't introduced errors in your code**. It just means that git was able to mix and match your contributions without needing your aid because the changes were done in different lines of the same file.

Let's now play with the case where conflicts arise...

### Conflict resolution & peacekeeping

<img src="conflict.svg" height="400" align="center"/>

From: http://swcarpentry.github.io/git-novice/09-conflict/index.html

Beneath the apparent simplicity, merging branches is a complicated task. Git can use different algorithms to decide how to perform the merge depending on the type of divergences that the two branches have. Explaining the merge strategies is out of the scope of this course (and a very complicated topic, see for instance [here](http://blog.plasticscm.com/2011/09/merge-recursive-strategy.html) and [here](http://raulavila.com/2017/03/como-funciona-git-3/)).

For now we will content ourselves with creating a situation where we have conflicts and learning how to resolve them:

In [ ]:
%%bash
# we will emulate the situation where someone else has started working on a feature off
# the master branch, and she changed a line that we also changed. In this case
# the change is a bit silly (a variable name change), though perhaps not a completely
# unfamiliar situation :)
cd my_first_repo/
git checkout -b super_feature master

**Exercise 3.1**: Please introduce a **conflict** with respect to our `$USER/merge_work` branch, by making changes in one of the lines that you previously modified in either one of the branches you merged.
(We can loosely define a conflict as a change in the same line of the same file, starting from the same parent commit)

Now if we try to merge this `super_feature` branch with our `$USER/merge_work`... 

In [ ]:
%%bash
cd my_first_repo/
git diff super_feature $USER/merge_work

In [ ]:
%%bash
cd my_first_repo/
git merge $USER/merge_work

Uh oh! Conflicts happened! No need to freak out though because git is nice enough to tell us what to do next:

```
CONFLICT (content): Merge conflict in plot_compare_reduction.py
Automatic merge failed; fix conflicts and then commit the result.
```

So let's do it, we will open the `plot_compare_reduction.py` file and see where `git` run into trouble:

In [ ]:
%%bash
cd my_first_repo/
cat plot_compare_reduction.py

**Exercise 2.0:** Fix the merge conflicts and commit the changes. The merge is now done and we can move on to the next section!

### [Advanced] Rebasing

WIP

## Remote repositories & teleportation

As you know, a lot of the work we Data Scientists do is not limited to being developed and run in our local machine. In addition, we also like our work to be safe and secure somewhere outside the reach of a potential coffee spill...

Remote repositories are versions of our local repository that are stored... remotely (hence the teleportation pun). The most well-known remote server is `github.com` but you should know there's a few other well known ones like `bitbucket`, `sourceforge` or `gitlab`. 

In this section we will see how to perform the most common operations with remote repositories:
* Creating a remote repository and setting up a remote for your local repository
* Listing and manipulating the remote repositories for an existing project (remote)
* Pushing and pulling from remote repositories (fetch, pull, push)
* Issuing Pull Requests 

Reference: https://git-scm.com/book/en/v2/Git-Basics-Working-with-Remotes

### Creating a remote repository and setting up a remote for your local repository

If we try to see what remotes are currently set up for the repository we have created in earlier parts of this tutorial, we see that there's none:

In [ ]:
%%bash
cd my_first_repo/
git remote -v

So since we want to be able to save and share all our hard work with other people, our first step will be to add a remote to `my_first_repo`.

Start by creating a new public repository on [github.com](https://github.com/). For that you will need a github user. Please name your repository `my_first_repo`. After that, you can add this remote to your repo as follows:

In [ ]:
%%bash
cd my_first_repo/
git remote -v
git remote add luca git@github.com:atibaup/my_second_repo.git

So now if you try again, you should see your newly added remote:

In [ ]:
%%bash
cd my_first_repo/
git remote -v

**Exercise 2.1**: Pair up with another student and add his repository as one of your remotes.

If you want to see the branches for a remote, it suffices to run

In [ ]:
%%bash
cd my_first_repo/
git remote show luca 
# In this case you won't see much information here because the remote repository you created only has one branch

Now that your repository has a brand new remote, you can merrily pull and push data from and towards it. Since the remote repository is brand new, there's nothing for us yet to fetch:

In [ ]:
%%bash
cd my_first_repo/
git fetch luca

So we can push our local branches to the remote repository:

In [ ]:
%%bash
cd my_first_repo/
git push luca master

And voilà, you and everyone else (with access) can play with your world-saving contributions!

**Exercise 2.2**: Fetch from your colleague's remotes which you added in Exercise 2.1.

### Fetching is not pulling

When you fetch a branch from a remote, `git` only saves a copy of the remotes snapshot in your local repository, under a special type of branch called "remote-tracking branches" that explicitly relate them to the remote you fetched from.

This means that in order to incorporate the remote branch changes into your local branch, you would need to `merge` the content from the recently fetched branch into your local branch. So you can either:

In [ ]:
%%bash
cd my_first_repo/
# assuming you're already in your master branch
git fetch luca
# likely will do nothing because both your remote and local master branch are already in sync,
# unless you have added changes to either on your own


In [ ]:
%%bash
cd my_first_repo/
git log

In [ ]:
%%bash
cd my_first_repo/
git merge luca/master

Or use `git pull` synthactic sugar, which fetches and merges the remote-tracking branch into your current branch:

In [ ]:
%%bash
cd my_first_repo/
# assuming you're already in your master branch
git pull origin master

### Tracking branches

Tracking branches are local branches that are directly related to a remote branch. They are convenient because they save us some typing when we want to pull/push from a remote branch so that instead of having to write:
```
git pull my_remote my_remote_branch
```
we can simply do:
```
git pull
```
To create a tracking branch directly from an existing remote branch, git offers the following shortcut:
```
git checkout --track origin/serverfix
```
instead of the more verbose `git checkout -b [branch] [remotename]/[branch]`.

On the other hand, if you want to quickly see what if any remote branches your local branch are tracking, you can run:

In [ ]:
%%bash
cd my_first_repo/
git branch -vv

If you want an existing local branch to start tracking a specific remote branch,
it suffices to run ```git branch -u origin/serverfix```

### Issuing pull requests

When working in a collaborative environment, you don't usually want to push your contributions directly to a repository's master branch. One of the reasons is that the code you modified is probably not just your own, which means you may want to get it double-checked by a more knowledgeable pair of eyes. Another reason is that it is usually good practice to let someone else be aware of the changes you introduced, in case you decide to take that long-awaited tropical beach vacation before everything breaks down...

There are some good practices that are helpful to make the most of code reviews during pull request.

For the contributor:
1. Make your code reproducible and provide reproducibility instructions: it's easier to understand things if one can run them!
2. Describe the purpose of the code changes in the PR description
3. Give instructions on what you would like to have reviewed (ex: a specific function, a line of code that you're not sure about)
4. Make small pull requests! (ex: [The art of the small pull request](https://medium.com/letgo/the-art-of-the-small-pull-request-303f7ef63901))
5. Always assign your PR, if possible to one or two reviewers, not more.


For the reviewer:
1. Review the PR promptly, or else let the contributor know if you can't.
2. Be nice! Everyone writes bad code, and most of the times, it's ok!
3. Categorize comments as "minor" (not necessary for PR approval) or "required".
4. Try to learn from other people's brains.


If you are curious, these are a couple of interesting additional references regarding collaboration workflows in git:
* https://reflectoring.io/github-fork-and-pull/
* https://www.atlassian.com/git/tutorials/comparing-workflows
